In [2]:
import importlib
import os
from pathlib import Path
import sys

from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.gis import GIS
from dotenv import load_dotenv, find_dotenv
import pandas as pd

# import arcpy if available
if importlib.util.find_spec("arcpy") is not None:
    import arcpy

In [7]:
# paths to common data locations - NOTE: to convert any path to a raw string, simply use str(path_instance)
project_parent = Path('./').absolute().parent

data_dir = project_parent/'data'

data_raw = data_dir/'raw'
data_ext = data_dir/'external'
data_int = data_dir/'interim'
data_out = data_dir/'processed'

gdb_raw = data_raw/'raw.gdb'
gdb_int = data_int/'interim.gdb'
gdb_out = data_out/'processed.gdb'

# import the project package from the project package path
sys.path.insert(0, str(project_parent/'src'))
import dm

# load the "autoreload" extension so that code can change, & always reload modules so that as you change code in src, it gets loaded
%load_ext autoreload
%autoreload 2

# load environment variables from .env
load_dotenv(find_dotenv())

# create a GIS object instance; if you did not enter any information here, it defaults to anonymous access to ArcGIS Online
gis = GIS(
    url=os.getenv('ESRI_GIS_URL'), 
    username=os.getenv('ESRI_GIS_USERNAME'),
    password=os.getenv('ESRI_GIS_PASSWORD')
)

gis

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


GIS @ https://geoai.maps.arcgis.com version:8.2

In [9]:
gdb = dm._registry.get_ba_demographic_gdb_path()

gdb

WindowsPath('D:/arcgis/ba_data/Data/Demographic Data/USA_ESRI_2019.gdb')

In [25]:
fc = gdb/'BlockGroups_bg'
aoi_fc = gdb/'CBSAs_cb'
aoi_selector = 'Seattle'

In [30]:
lyr = arcpy.management.MakeFeatureLayer(str(fc))[0]

arcpy.management.GetCount(lyr)

<Result '217209'>

In [27]:
aoi_lyr = arcpy.management.MakeFeatureLayer(str(aoi_fc), where_clause=f"NAME LIKE '%{aoi_selector}%'")[0]

In [28]:
sel_lyr = arcpy.management.SelectLayerByLocation(
    in_layer=lyr,
    overlap_type='HAVE_THEIR_CENTER_IN',
    select_features=aoi_lyr
)[0]

In [29]:
arcpy.management.GetCount(sel_lyr)

<Result '2474'>

In [31]:
sel_df = GeoAccessor.from_featureclass(sel_lyr, fields=['ID'])

sel_df.head()

,ID,SHAPE
0,530530701003,"{""rings"": [[[-13587576.896200001, 5931283.3849..."
1,530530714071,"{""rings"": [[[-13618997.0451, 5953796.150899999..."
2,530530714072,"{""rings"": [[[-13621890.9066, 5953114.284599997..."
3,530530714073,"{""rings"": [[[-13622599.9004, 5953136.347999997..."
4,530530714112,"{""rings"": [[[-13627506.4183, 5953782.585000001..."


In [32]:
sel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2474 entries, 0 to 2473
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   ID      2474 non-null   object  
 1   SHAPE   2474 non-null   geometry
dtypes: geometry(1), object(1)
memory usage: 38.8+ KB
